In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))  # sobe um nível a partir da pasta models/

In [2]:
import ast
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

from copy import deepcopy
from sklearn.model_selection import KFold

from val.Wrapper import ForwardFeatureSelector

In [3]:
class MLP_100_100_100_classifier(nn.Module):
    def __init__(self, input_len):
        super().__init__()

        self.layer1 = nn.Linear(input_len, 100)  
        self.relu = nn.ReLU()                          
        self.hidden_layer_relu = nn.Sequential(
            nn.Linear(100,100),
            nn.ReLU(),
            nn.Linear(100,100),
            nn.ReLU(),
            nn.Linear(100,100),
            nn.ReLU()
        )
        self.layer2 = nn.Linear(100, 30)

    def forward(self, x):

        x = self.relu(self.layer1(x))
        x = self.hidden_layer_relu(x)
        logits = self.layer2(x)
        
        return logits

In [4]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

y_best_ansatz = np.argmax(y, axis=1)  

# criando o kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [6]:
# implementação de um critério de parada para o modelo parar no ponto "ótimo"
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [7]:
# 3. Função para treinar um fold
def train_fold(model, train_loader, val_loader, criterion, optimizer, n_epochs=1000000, patience = 10, min_delta = 0.0):
    model.to(device)
    early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)
    best_model = None
    best_acc = 0.0
    
    for epoch in range(n_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze()  # Converte (batch_size, 1) para (batch_size,)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validação
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            total = 0
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                y_val = y_val.squeeze()  # Garante que y_val é 1D
                
                outputs = model(X_val)
                val_loss += criterion(outputs, y_val).item()  
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == y_val).sum().item()
                total += y_val.size(0)
        
        
        val_acc = correct / total
        val_loss /= len(val_loader)
        train_loss /= len(train_loader)
        
        print(f'Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.4f}')
        
        # Early Stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
        # Salvar melhor modelo
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = deepcopy(model.state_dict())
    
    return best_model, best_acc

In [8]:
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n=== Fold {fold + 1}/{3} ===")
    
    # Dividir e preparar dados
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_best_ansatz[train_idx], y_best_ansatz[val_idx]
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    
    # Inicializar modelo e otimizador
    model = MLP_100_100_100_classifier(22)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01) 
    
    # Treinar fold
    best_model, best_acc = train_fold(model, train_loader, val_loader, criterion, optimizer, patience=15)
    results.append(best_acc)
    
    # Salvar modelo se necessário
    torch.save(best_model, f'./models_salvos/best_model_MLP-100-100-100-classifier_fold{fold+1}.pt')

# 7. Resultados finais
print("\n=== Resultados ===")
print(f"Acurácia média: {np.mean(results):.4f} ± {np.std(results):.4f}")
print(f"Acurácias por fold: {results}")


=== Fold 1/3 ===
Epoch 1/1000000 - Train Loss: 3.0220 - Val Loss: 3.0197 - Val Acc: 0.0583
Epoch 2/1000000 - Train Loss: 2.6556 - Val Loss: 2.8120 - Val Acc: 0.3000
Epoch 3/1000000 - Train Loss: 2.4528 - Val Loss: 2.6871 - Val Acc: 0.3000
Epoch 4/1000000 - Train Loss: 2.4343 - Val Loss: 2.7326 - Val Acc: 0.3000
Epoch 5/1000000 - Train Loss: 2.3633 - Val Loss: 2.6949 - Val Acc: 0.2750
Epoch 6/1000000 - Train Loss: 2.2957 - Val Loss: 2.7392 - Val Acc: 0.2583
Epoch 7/1000000 - Train Loss: 2.2756 - Val Loss: 2.7175 - Val Acc: 0.2583
Epoch 8/1000000 - Train Loss: 2.2801 - Val Loss: 2.7042 - Val Acc: 0.2583
Epoch 9/1000000 - Train Loss: 2.2597 - Val Loss: 2.6920 - Val Acc: 0.2583
Epoch 10/1000000 - Train Loss: 2.2276 - Val Loss: 2.7048 - Val Acc: 0.2583
Epoch 11/1000000 - Train Loss: 2.2334 - Val Loss: 2.7191 - Val Acc: 0.2583
Epoch 12/1000000 - Train Loss: 2.2058 - Val Loss: 2.7372 - Val Acc: 0.2750
Epoch 13/1000000 - Train Loss: 2.2482 - Val Loss: 2.7284 - Val Acc: 0.3000
Epoch 14/1000000

In [9]:
model = MLP_100_100_100_classifier(22).to(device)
#carregando o melhor fold treinado
model.load_state_dict(torch.load('.//models_salvos/best_model_MLP-100-100-100-classifier_fold2.pt'))  # Substitua pelo caminho correto
model.eval()  # Modo de avaliação

MLP_100_100_100_classifier(
  (layer1): Linear(in_features=22, out_features=100, bias=True)
  (relu): ReLU()
  (hidden_layer_relu): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
  )
  (layer2): Linear(in_features=100, out_features=30, bias=True)
)

In [10]:
input = X[6]
input_tensor = torch.FloatTensor(input).to(device)

with torch.no_grad():
    logits = model(input_tensor)
    print(logits)
    print(logits.shape)
    probabilities = torch.softmax(logits, dim=0)  # Shape (1, 30)

tensor([ 2.3737e+00,  3.2321e-03,  1.9258e+00, -3.9920e+00, -7.8860e+00,
        -7.2705e+00,  2.1764e+00,  1.8194e+00, -1.3945e+00, -5.6717e+00,
        -3.7648e+00,  7.2861e-01,  8.3512e-01,  1.8855e-01,  2.1325e+00,
        -7.4403e+00, -9.5926e-01, -3.9527e+00, -4.9849e+00, -4.1674e+00,
        -3.9788e+00,  4.2663e-02, -3.6794e+00, -5.0568e+00, -4.3920e+00,
        -3.2086e-01, -4.9165e+00, -4.1407e+00, -5.8382e+00, -4.1937e+00])
torch.Size([30])


In [11]:
print(probabilities)

tensor([2.1390e-01, 1.9986e-02, 1.3667e-01, 3.6782e-04, 7.4898e-06, 1.3861e-05,
        1.7560e-01, 1.2288e-01, 4.9396e-03, 6.8570e-05, 4.6162e-04, 4.1281e-02,
        4.5921e-02, 2.4055e-02, 1.6806e-01, 1.1696e-05, 7.6334e-03, 3.8256e-04,
        1.3627e-04, 3.0863e-04, 3.7269e-04, 2.0790e-02, 5.0277e-04, 1.2682e-04,
        2.4654e-04, 1.4453e-02, 1.4592e-04, 3.1697e-04, 5.8056e-05, 3.0061e-04])


In [12]:
# Obter índices e valores das top-k probabilidades (ex.: top-5)
k = 3
top_k_probs, top_k_indices = torch.topk(probabilities, k=k, dim=0)

# Converter para numpy e exibir
top_k_probs = top_k_probs.cpu().numpy().flatten()
top_k_indices = top_k_indices.cpu().numpy().flatten()

print("Melhores ansatzes e suas probabilidades:")
for i, (idx, prob) in enumerate(zip(top_k_indices, top_k_probs)):
    print(f"{i+1}º: Ansatz {idx} - Probabilidade: {prob:.4f}")

Melhores ansatzes e suas probabilidades:
1º: Ansatz 0 - Probabilidade: 0.2139
2º: Ansatz 6 - Probabilidade: 0.1756
3º: Ansatz 14 - Probabilidade: 0.1681


In [13]:
from sklearn.metrics import accuracy_score

selector = ForwardFeatureSelector(
    model=lambda input_dim: MLP_100_100_100_classifier(input_dim),
    model_type='pytorch',
    scoring=accuracy_score,
    cv=3,
    verbose=1,
    pytorch_optimizer= optim.Adam,
)

In [14]:
X_new = selector.fit_transform(X, y_best_ansatz)

Testing feature set: [0]


Testing feature set: [1]
Testing feature set: [2]
Testing feature set: [3]
Testing feature set: [4]
Testing feature set: [5]
Testing feature set: [6]
Testing feature set: [7]
Testing feature set: [8]
Testing feature set: [9]
Testing feature set: [10]
Testing feature set: [11]
Testing feature set: [12]
Testing feature set: [13]
Testing feature set: [14]
Testing feature set: [15]
Testing feature set: [16]
Testing feature set: [17]
Testing feature set: [18]
Testing feature set: [19]
Testing feature set: [20]
Testing feature set: [21]
Selected feature: 13 | Score: 0.3389
Testing feature set: [13, 0]
Testing feature set: [13, 1]
Testing feature set: [13, 2]
Testing feature set: [13, 3]
Testing feature set: [13, 4]
Testing feature set: [13, 5]
Testing feature set: [13, 6]
Testing feature set: [13, 7]
Testing feature set: [13, 8]
Testing feature set: [13, 9]
Testing feature set: [13, 10]
Testing feature set: [13, 11]
Testing feature set: [13, 12]
Testing feature set: [13, 14]
Testing feature 